# Use NLP and Naive Bayes to predict an email is Ham or Spam

In [1]:
import pyspark
from pyspark import SparkContext

sc = SparkContext(master='local', appName = 'HamvsSpam')

In [2]:
import findspark
findspark.init()
import pyspark
findspark.find()

'/Users/vunguyen/server/spark-2.4.6-bin-hadoop2.7'

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
'''
conf= pyspark.SparkConf().setAppName('tets_app').setMaster('local')
sc=pyspark.SparkContext(conf=conf)
sc
'''

#sc= SparkContext()
spark = (SparkSession
         .builder
         .appName('HamvsSpam')
         .getOrCreate())
spark

In [4]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [6]:
data = spark.read.csv('File/smsspamcollection/SMSSpamCollection', inferSchema=True, sep='\t')

In [7]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [8]:
data.show(5)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
+-----+--------------------+
only showing top 5 rows



In [9]:
from pyspark.sql.functions import length

In [10]:
data = data.withColumn('length',length(data['text']))

In [11]:
data.show(5)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
+-----+--------------------+------+
only showing top 5 rows



In [12]:
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [17]:
from pyspark.ml.feature import *

In [18]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')

In [20]:
# 'StopWordsRemover_e7366f895d81 parameter locale given invalid value en_VN.'
locale = sc._jvm.java.util.Locale
locale.setDefault(locale.forLanguageTag("en-US"))

stopremove = StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')

In [21]:
count_vec = CountVectorizer(inputCol='stop_tokens', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_num = StringIndexer(inputCol='class', outputCol='label')

In [24]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes()

In [23]:
from pyspark.ml import Pipeline

In [32]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

clean_up = VectorAssembler(inputCols=['tf_idf','length'], 
                           outputCol='features')

In [33]:
data_prep_pipe = Pipeline(stages = [ham_spam_to_num,
                                   tokenizer,
                                   stopremove,
                                   count_vec,
                                   idf,
                                   clean_up])

In [34]:
cleaner = data_prep_pipe.fit(data)

In [35]:
clean_data = cleaner.transform(data)

In [36]:
clean_data = clean_data.select(['label','features'])

In [37]:
clean_data.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
+-----+--------------------+
only showing top 10 rows



In [38]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [39]:
spam_predictor = nb.fit(training)

In [40]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [41]:
test_results = spam_predictor.transform(testing)

In [42]:
test_results.show(10)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-793.72240653313...|[1.0,9.3853309491...|       0.0|
|  0.0|(13424,[0,1,2,13,...|[-627.31668383829...|[1.0,2.8223080762...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[-1072.6996864515...|[1.0,1.6216083377...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-850.66969408604...|[1.0,1.9644640080...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-534.49244884595...|[1.0,3.9096178250...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-691.39867270743...|[1.0,2.4097302283...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-688.18730994813...|[1.0,1.2458547718...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-851.98814518421...|[1.0,9.6354554329...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-969.31788475159...|[1.0,4.8853240714...|       0.0|
|  0.0|(13424,[0

In [43]:
test_results.groupby('label','prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  227|
|  0.0|       1.0|  141|
|  1.0|       0.0|    5|
|  0.0|       0.0| 1335|
+-----+----------+-----+



In [48]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [50]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)

In [51]:
print('Accuracy: {}'.format(acc))

Accuracy: 0.9221435002838692
